In [1]:
import formulas
import utilities
import pandas as pd
import os
import re


def get_formulas_df(formulas_input_df, sort_orders_df): 
    
    formula_sheet_list = formulas.get_formulas_on_formula_sheet(
        formulas_input_df)
    by_year_df =  (
        formulas.get_formulas_by_year_df(formulas_input_df))
    cumulative_df = (
        formulas.get_formulas_by_year_cumulative_df(
            formulas_input_df, sort_orders_df))
    formulas_df = pd.concat([by_year_df, cumulative_df])
    return(formulas_df)



def create_formulas_content(formulas_df, formula_sheet_list, 
                            sort_orders_df, docs_dir):

    dirs_df = (
        formulas_df[['State', 'Formula sub category 1', 
                             'Formula sub category 2',
                             'Subject code']].drop_duplicates())
    file_paths_df = (
        formulas_df[['State', 'Formula sub category 1', 
                             'Formula sub category 2',
                             'Subject code', 'Category']].drop_duplicates())
    
    utilities.create_sub_directories_from_df(base_dir = docs_dir, 
                                             sub_paths_df = dirs_df)
    
    front_matter_index_files = {'bookCollapseSection' : True}
    utilities.create_index_files(base_dir=docs_dir, dirs_df=dirs_df, 
                       front_matter=front_matter_index_files,
                       sort_orders_df = sort_orders_df)
    
    utilities.create_files(base_dir = docs_dir, file_paths_df= file_paths_df, 
                           file_extension='.md', 
                           fn=formulas.get_formula_display_string, 
                           sort_orders_df = sort_orders_df,
                           formulas_df = formulas_df, 
                           formula_sheet_list = formula_sheet_list, 
                           cols_to_highlight_if_in_formula_sheet = [
                               'Formula_1', 'Formula_2'])


def create_calculus_summary(formulas_df, formula_sheet_list,
                            sort_orders_df, docs_dir):
    
    calculus_summary_dirs_df = (
        formulas.get_calculus_summary_dir_paths_df(formulas_df))
    utilities.create_sub_directories_from_df(
        base_dir = docs_dir, sub_paths_df = calculus_summary_dirs_df)    
    front_matter_index_files = {'bookCollapseSection' : True}
    utilities.create_index_files(
        base_dir=docs_dir, dirs_df=calculus_summary_dirs_df, 
        front_matter=front_matter_index_files,
        sort_orders_df = sort_orders_df)

    calculus_summary_file_paths_df = (
        formulas.get_calculus_summary_file_paths_df(calculus_summary_dirs_df))

    utilities.create_files(base_dir = docs_dir, 
                           file_paths_df= calculus_summary_file_paths_df, 
                           file_extension='.md', 
                           fn=formulas.get_calculus_summary_display_string, 
                           sort_orders_df = sort_orders_df,
                           formulas_df = formulas_df, 
                           formula_sheet_list = formula_sheet_list, 
                           cols_to_highlight_if_in_formula_sheet = [
                               'Differentiation', 'Integration'])
    

def create_financial_summary(formulas_df, formula_sheet_list,
                            sort_orders_df, docs_dir):
    
    financial_summary_dirs_df = (
        formulas.get_financial_summary_dir_paths_df(formulas_df))
    utilities.create_sub_directories_from_df(
        base_dir = docs_dir, sub_paths_df = financial_summary_dirs_df)    
    front_matter_index_files = {'bookCollapseSection' : True}
    utilities.create_index_files(
        base_dir=docs_dir, dirs_df=financial_summary_dirs_df, 
        front_matter=front_matter_index_files,
        sort_orders_df = sort_orders_df)

    financial_summary_file_paths_df = (
        formulas.get_financial_summary_file_paths_df(
            financial_summary_dirs_df))

    utilities.create_files(base_dir = docs_dir, 
                           file_paths_df= financial_summary_file_paths_df, 
                           file_extension='.md', 
                           fn=formulas.get_financial_summary_display_string, 
                           sort_orders_df = sort_orders_df,
                           formulas_df = formulas_df, 
                           formula_sheet_list = formula_sheet_list, 
                           cols_to_highlight_if_in_formula_sheet = [
                               'Arithmetic sequence', 'Geometric sequence'])

if __name__ == '__main__':

    website_creator_dir = os.getcwd()
    docs_dir = utilities.get_docs_path(website_creator_dir)
    formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv'
    order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'

    formulas_input_df = pd.read_csv(formula_file_path)
    sort_orders_df = pd.read_csv(order_file_path)
    formulas_df = get_formulas_df(formulas_input_df, sort_orders_df)
    formula_sheet_list = (
        formulas.get_formulas_on_formula_sheet(formulas_input_df))
    
    utilities.delete_directory_if_it_exists(docs_dir)
    create_formulas_content(formulas_df, formula_sheet_list, 
                            sort_orders_df, docs_dir)
    create_calculus_summary(formulas_df, formula_sheet_list, 
                            sort_orders_df, docs_dir)
    create_financial_summary(formulas_df, formula_sheet_list, 
                            sort_orders_df, docs_dir)


In [4]:
import os
import formulas
import utilities
import pandas as pd

website_creator_dir = os.getcwd()
docs_dir = utilities.get_docs_path(website_creator_dir)
formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv'

formulas_input_df = pd.read_csv(formula_file_path)



,ID,State,Category,Sub-category_1,Sub-category_2,Description,Group,Formula_1,Formula_2,Subject code,On formula sheet,Comment
0,1,NSW,Indices,Index Laws,NaN,Multiplying terms with same base,NaN,$a^m \times a^n = a^{m+n}$,NaN,Year 9,False,NaN
1,2,NSW,Indices,Index Laws,NaN,Dividing terms with same base,NaN,$a^m \div a^n = \dfrac{a^m}{a^n} = a^{m-n}$,NaN,Year 9,False,NaN
2,3,NSW,Indices,Index Laws,NaN,Power of a power,NaN,$(a^m)^n = a^{m \times n}$,NaN,Year 9,False,NaN
3,4,NSW,Indices,Index Laws,NaN,Powers of products,NaN,$(ab)^n = a^nb^n$,NaN,Year 9,False,NaN
4,5,NSW,Indices,Index Laws,NaN,Powers of quotients,NaN,$\left(\dfrac{a}{b}\right)^n = \dfrac{a^n}{b^n}$,NaN,Year 9,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
67,69,NSW,Financial mathematics,Arithmetic sequence,Sum of first n terms,Sum of the first n terms of an arithmetic sequ...,NaN,$S_n = \dfrac{n}{2}[2a + (n-1)d] = \dfrac{n}{2...,NaN,Year 12 Adv,True,NaN
68,70,NSW,Financial mathematics,Geometric sequence,Recursive definition,Recursive definition of a geometric sequence,NaN,$T_n = rT_{n-1}$,NaN,Year 12 Adv,False,NaN
69,71,NSW,Financial mathematics,Geometric sequence,n-th term,n-th term of a geometric sequence,NaN,$T_n = ar^{n-1}$,NaN,Year 12 Adv,True,NaN
70,72,NSW,Financial mathematics,Geometric sequence,Sum of first n terms,Sum of the first n terms of a geometric sequence,NaN,$S_n = \dfrac{a(1-r^n)}{1-r} = \dfrac{a(r^n-1)...,NaN,Year 12 Adv,True,NaN


In [1]:
# Retrieving a single formula
import formulas
import os
import pandas as pd

website_creator_dir = os.getcwd()
formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv'
formulas_input_df = pd.read_csv(formula_file_path)

result  = formulas.get_single_formula_1(
    formulas_input_df, 'Financial mathematics', 
    'Recursive definition of an arithmetic sequence')

display(result)

<IPython.core.display.Math object>

In [3]:
! pwd

/home/charl/Onedrive/Documents_Charl/Computer_Technical/Programming_GitHub/AustralianSchoolMaths/WebsiteCreator
